# 데이터 블록 만드는 법 (1)

> fastai에서는 데이터를 정의하는 방법으로 DataBlock API를 제안합니다. 각 인자가 의미하는 내용과, 실제 Siamese 공식 튜토리얼에 이 내용이 어떻게 적용되는지를 살펴봅니다.
- author: "Chansung Park"
- toc: true
- image: images/datablock/siamese-block.png
- comments: true
- categories: [datablock, siamese, fastai]
- permalink: /datablock-siamese/
- badges: false
- search_exclude: true

In [1]:
#hide
!pip install fastai
!pip install nbdev

     |████████████████████████████████| 188 kB 1.8 MB/s eta 0:00:01
     |████████████████████████████████| 108.1 MB 6.1 kB/s eta 0:00:011
     |████████████████████████████████| 48 kB 8.2 MB/s  eta 0:00:01
     |████████████████████████████████| 7.2 MB 63 kB/s s eta 0:00:01
     |████████████████████████████████| 10.1 MB 7.2 MB/s eta 0:00:01
     |████████████████████████████████| 1.0 MB 9.2 MB/s eta 0:00:01
     |████████████████████████████████| 2.2 MB 9.9 MB/s eta 0:00:01
     |████████████████████████████████| 8.5 MB 74 kB/s  eta 0:00:01
  Using cached future-0.18.2.tar.gz (829 kB)
     |████████████████████████████████| 301 kB 10.5 MB/s eta 0:00:01
     |████████████████████████████████| 287 kB 8.9 MB/s eta 0:00:01
  Using cached blis-0.4.1-cp38-cp38-macosx_10_9_x86_64.whl (3.7 MB)
  Using cached plac-1.1.3-py2.py3-none-any.whl (20 kB)
  Using cached catalogue-1.0.0-py2.py3-none-any.whl (7.7 kB)
     |████████████████████████████████| 71 kB 7.1 MB/s  eta 0:00:01
     |███████████

  Attempting uninstall: nbconvert
    Found existing installation: nbconvert 6.0.6
    Uninstalling nbconvert-6.0.6:
      Successfully uninstalled nbconvert-6.0.6


In [20]:
#hide
from fastai.vision.all import *
import nbdev

In [14]:
#hide
# path = untar_data(URLs.PETS)
files = ""

def label_func(fname):
    return re.match(r'^(.*)_\d+.jpg$', fname.name).groups()[0]

class ImageTuple(fastuple):
    @classmethod
    def create(cls, fns): return cls(tuple(PILImage.create(f) for f in fns))
    
    def show(self, ctx=None, **kwargs): 
        t1,t2 = self
        if not isinstance(t1, Tensor) or not isinstance(t2, Tensor) or t1.shape != t2.shape: return ctx
        line = t1.new_zeros(t1.shape[0], t1.shape[1], 10)
        return show_image(torch.cat([t1,line,t2], dim=2), ctx=ctx, **kwargs)

def ImageTupleBlock():
    return TransformBlock(type_tfms=ImageTuple.create, batch_tfms=IntToFloatTensor)

# splits = RandomSplitter()(files)
# splits_files = [files[splits[i]] for i in range(2)]
# splits_sets = mapped(set, splits_files)

def get_split(f):
    for i,s in enumerate(splits_sets):
        if f in s: return i
    raise ValueError(f'File {f} is not presented in any split.')

# splbl2files = [{l: [f for f in s if label_func(f) == l] for l in labels} for s in splits_sets]

def splitter(items): 
    def get_split_files(i): 
        return [j for j,(f1,f2,same) in enumerate(items) if get_split(f1)==i]

    return get_split_files(0),get_split_files(1)

def draw_other(f):
    same = random.random() < 0.5

    cls = label_func(f)
    split = get_split(f)

    if not same: 
        cls = random.choice(L(l for l in labels if l != cls)) 

    return random.choice(splbl2files[split][cls]), same

def get_tuples(files): 
    return [[f, *draw_other(f)] for f in files]

def get_x(t): 
    return t[:2]

def get_y(t): 
    return t[2]

In [23]:
nbdev.show_doc(DataBlock)

<h2 id="DataBlock" class="doc_header"><code>class</code> <code>DataBlock</code><a href="https://github.com/fastai/fastai/tree/master/fastai/data/block.py#L58" class="source_link" style="float:right">[source]</a></h2>

> <code>DataBlock</code>(**`blocks`**=*`None`*, **`dl_type`**=*`None`*, **`getters`**=*`None`*, **`n_inp`**=*`None`*, **`item_tfms`**=*`None`*, **`batch_tfms`**=*`None`*, **`get_items`**=*`None`*, **`splitter`**=*`None`*, **`get_y`**=*`None`*, **`get_x`**=*`None`*)

Generic container to quickly build `Datasets` and `DataLoaders`

**DataBlock**은 모델이 학습하는 데이터를 준비시키는 핵심 API 입니다. 중요한 사실은 **DataBlock**은 일종의 **템플릿** 이라는 것입니다. 실제로 데이터가 **주입되었을 때**, **'이런 이런 식으로 동작한다'** 를 정의하는 것이죠.

상기 **DataBlock** API의 원형을 말로 풀어서 설명하면 다음과 같습니다.

- **dls = siamese.dataloaders(files)**
  - 이 부분은 DataBlock을 만든 후 수행되는 코드입니다.
  - 입력된 것은 1이 수용할 데이터 목록입니다.


- **get_items**: 수용된 데이터를 논리적인 집합 목록으로 만듭니다.
  - 이 함수는 무엇을 반환해도 상관이 없습니다. 단, 반환된 값으로 입력될 데이터와 레이블의 추출이 가능해야 합니다.
  - 이 단계에서 다뤄지는 **데이터**란, 아무런 변환 처리가 적용되지 않은 **raw** 입니다.
    - *예시 1) 이미지 파일 경로의 목록 (경로에서 입력으로 사용될 파일이름과, 상위폴더로 결정 가능한 레이블이 모두 포함되어 있다면 OK)*
    - *예시 2) (이미지 파일 경로 1, 이미지 파일 경로 2, 레이블) 튜플들의 목록*


- **get_x**: 무엇을 입력으로 삼을지 결정합니다.
  - **get_items**이 반환한 목록을 하나씩 접근해서 처리합니다.
  - 이 단계에서도 다뤄지는 데이터는 raw 입니다.
    - *예시 1) 이미지 파일 경로를 그대로 반환 (bypass)*
    - *예시 2) (이미지 파일 경로 1, 이미지 파일 경로 2, 레이블) 튜플에서 1과 2를 추출해서 반환*


- **get_y**: 무엇을 레이블로 삼을지 결정합니다. 
  - **get_items**이 반환한 목록을 하나씩 접근해서 처리합니다.
  - 이 단계에서도 다뤄지는 데이터는 raw 입니다.
    - *예시 1) 이미지 파일 경로에서 상위 폴더명을 추출해서 반환*
    - *예시 2) (이미지 파일 경로 1, 이미지 파일 경로 2, 레이블) 튜플에서 3을 추출해서 반환*


- **blocks**: raw 형식의 데이터를 모델에 입력 가능한 형식으로 바꿀 규칙을 결정합니다.
  - 두 개 이상을 지정할 수 있습니다. **get_x** 및 **get_y** 의 내용을 모두 수용할 수 있어야 합니다. 
  - 입력과 출력을 구분하기 위한 목적으로 **n_inp** 라는 인자를 건드릴 수 있습니다.
    - *예시 1) 이미지 파일 경로에서, 이미지를 불러오고 tensor로 변환하는 ImageBlock. 상위 폴더명을 원-핫 인코딩된 tensor로 변환하는 CategoryBlock*
    - *예시 2-1) 이미지 파일 두 개를 수용하기 위한, 두 개의 ImageBlock. 해당 레이블을 원-핫 인코딩 tensor로 변환하는 CategoryBlock*
    - *예시 2-2) 이미지 파일 두 개를 한번에 수용 가능한 TupleBlock. 해당 레이블을 원-핫 인코딩 tensor로 변환하는 CategoryBlock*
     

- **item_tfms**: 각 데이터 하나에 대한 변형을 하고 싶다면, 그 변형 규칙을 지정합니다.


- **batch_tfms**: 배치 단위의 데이터에 대한 변형을 하고 싶다면, 그 변형 규칙을 지정합니다.
  - 배치 단위의 데이터 변형은 배치 단위로 GPU(가용하다면) 에서 수행됩니다.
   
- **splitter**: 학습/검증 데이터셋을 구분하는 방법을 결정합니다.
  - 두 개(학습/검증)를 포함한 튜플 형식을 반환합니다.
  - 각각 리스트이며, 리스트에는 인덱스 목록이 들어 있습니다. **get_items** 에서 누구 인덱스를 학습용으로, 누구 인덱스를 검증용으로 둘 것인지를 나열한 것입니다.

In [15]:
siamese = DataBlock(   
    get_items=get_tuples,                       # 모든 데이터를 불러 들이는 함수를 지정합니다.
        get_x=get_x,                            # 불러와진 데이터에 대해서, 입력을 결정하는 함수를 지정합니다.
        get_y=get_y,                            # 불러와진 데이터에 대해서, 출력을 결정하는 함수를 지정합니다.
       blocks=(ImageTupleBlock, CategoryBlock), # tuple 형식으로, 두 개 이상도 가능합니다.
    item_tfms=Resize(224),                      # 아이템 단위의 변환
   batch_tfms=[Normalize.from_stats(*imagenet_stats)],   # 배치 단위의 변환
     splitter=splitter                         # 학습/검증 데이터셋을 분리하는 함수를 지정합니다.        
)

앞선 설명을 Siamese 데이터 블록에 대입해서 설명해 보자면

- **siamese.dataloaders(files)**
  - files는 단순히 Path 객체로 표현된 이미지 파일 경로의 목록 입니다.


- **get_items**: get_tuples
  - get_tuples 함수는 입력된 이미지 파일 경로 목록들로부터, 튜플 (이미지 파일 경로 1, 이미지 파일 경로 2, 레이블) 목록을 만들어 반환합니다. 
    - 우선 각 파일을 순차적으로 접근합니다. 접근될 때마다 무작위로 다른 파일을 하나 접근합니다.
    - 그리고 두 파일의 카테고리를 추출합니다. 두 파일의 카테고리가 일치한다면 True, 아니라면 False를 레이블로서 정합니다.
  
  
- **get_x**: get_x
  - get_x 함수는 **get_tuples** 가 반환한 목록을 하나씩 접근합니다.
    - 즉, (이미지 파일 경로 1, 이미지 파일 경로 2, 레이블) 튜플을 하나씩 처리합니다.
    - 해당 튜플 중 처음 두 개만이 입력 데이터이므로 튜플[:2] 를 반환합니다.
  
  
- **get_y**: get_y
  - get_y 함수는 **get_tuples** 가 반환한 목록을 하나씩 접근합니다.
    - 즉, (이미지 파일 경로 1, 이미지 파일 경로 2, 레이블) 튜플을 하나씩 처리합니다.
    - 해당 튜플 중 마지막 하나만이 출력 데이터이므로 튜플[2]를 반환합니다.


- **blocks**: (ImageTupleBlock, CategoryBlock)
  - ImageTupleBlock은 내부적으로 두 개의 이미지를 입력받고, 이들을 PILImage 형식으로 변환 후 TensorImage 형식으로 변환합니다.
    - ImageTupleBlock은 TransformBlock을 반환하는데, TransformBlock에는 type_tfms 및 batch_tfms 인자가 있습니다. type_tfms는 DataBlock의 item_tfms와 동일한 것으로 실제로는 머지되어 처리됩니다. 마찬가지로 TransformBlock에서 지정된 batch_tfms도 DataBlock의 batch_tfms와 머지되어 처리됩니다.
      - type_tfms에서 PILImage로의 변환 작업이 지정되고, batch_tfms에서 TensorImage로의 변환 작업이 지정되었습니다.
  - CategoryBlock은 주어진 레이블을 원-핫 인코딩된 tensor로 변환합니다.


- **item_tfms**: Resize(224)
   - 이미지 크기 조절은 Resize 라는 Transform 형 객체를 활용합니다.
   - item_tfms에 나열되는 Transform 류는 준비된 데이터 튜플(이미지 파일 경로 1, 이미지 파일 경로 2, 레이블)에 모두 적용됩니다.
     - 하지만, 내부적으로 자신이 적용 가능한 녀석이 아니면 그대로 종료됩니다. 즉, 이미지 파일 경로 1, 이미지 파일 경로 2에 대한 변환만이 수행됩니다.



- **batch_tfms**: [Normalize.from_stats(*imagenet_stats)]
   - **item_tfms** 때와 마찬가지로 적용될 대상을 스스로 알아챕니다
   - `Normalize.from_stats(*imagenet_stats)`는 이미지넷 학습에 사용된 데이터의 평균, 표준편차 값으로 전이학습 되는 데이터의 정규화를 수행한다는 의미를 가집니다. 그렇게 해서 전이학습 대상 모델이 학습한 데이터와 유사한 값들로 변환합니다.